In [1]:
library(tidyverse)
library(tigris)
library(sf)

options(tigris_use_cache = TRUE)
# https://opendata.fcc.gov/Wireline/Fixed-Broadband-Deployment-Data-June-2021-Status-V/jdr4-3q4p
fcc_data <- read.csv('../data/fcc.csv')

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.4.0     ✔ purrr   1.0.1
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.5.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

To enable caching of data, set `options(tigris_use_cache = TRUE)`
in your R script or .Rprofile.

Linking to GEOS 3.10.2, GDAL 3.4.3, PROJ 8.2.0; sf_use_s2() is TRUE



In [2]:
counties <- c('Asotin', 'Lincoln', 'Ferry', 'Garfield', 'Pend Oreille', 'Stevens', 'Whitman', 'Spokane',
    'Adams')

tract <- tracts(state = "washington", county = counties, progress_bar = FALSE, cb = FALSE)
block <- blocks(state = "washington", county = counties, progress_bar = FALSE)

Retrieving data for the year 2021

Retrieving data for the year 2021



In [3]:
head(tract)

Registered S3 method overwritten by 'geojsonsf':
  method        from   
  print.geojson geojson

Warning message in fun(libname, pkgname):
“rgeos: versions of GEOS runtime 3.10.2-CAPI-1.16.0
and GEOS at installation 3.8.0-CAPI-1.13.1differ”


,STATEFP,COUNTYFP,TRACTCE,GEOID,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<MULTIPOLYGON [°]>
14,53,075,000602,53075000602,6.02,Census Tract 6.02,G5020,S,83019278,0,+46.7712346,-117.0989449,MULTIPOLYGON (((-117.1716 4...
15,53,075,000601,53075000601,6.01,Census Tract 6.01,G5020,S,1028996,0,+46.7414143,-117.1645509,MULTIPOLYGON (((-117.1728 4...
16,53,075,000202,53075000202,2.02,Census Tract 2.02,G5020,S,2019222,0,+46.7387186,-117.1853588,MULTIPOLYGON (((-117.1965 4...
17,53,075,000201,53075000201,2.01,Census Tract 2.01,G5020,S,107067961,0,+46.7943975,-117.2247486,MULTIPOLYGON (((-117.3252 4...
53,53,063,011702,53063011702,117.02,Census Tract 117.02,G5020,S,7157584,205845,+47.6819109,-117.2370346,MULTIPOLYGON (((-117.272 47...
54,53,063,014400,53063014400,144,Census Tract 144,G5020,S,6308443,25162,+47.6933186,-117.3521333,MULTIPOLYGON (((-117.3639 4...


In [4]:
tract <- st_join(tract, block, left = FALSE)

In [5]:
names(tract)

[1] "STATEFP"    "COUNTYFP"   "TRACTCE"    "GEOID"      "NAME"      
 [6] "NAMELSAD"   "MTFCC"      "FUNCSTAT"   "ALAND"      "AWATER"    
[11] "INTPTLAT"   "INTPTLON"   "STATEFP20"  "COUNTYFP20" "TRACTCE20" 
[16] "BLOCKCE20"  "GEOID20"    "NAME20"     "MTFCC20"    "UR20"      
[21] "UACE20"     "UATYPE20"   "FUNCSTAT20" "ALAND20"    "AWATER20"  
[26] "INTPTLAT20" "INTPTLON20" "geometry"

In [6]:
head(fcc_data)

,Logical.Record.Number,Provider.ID,FRN,Provider.Name,DBA.Name,Holding.Company.Name,Holding.Company.Number,Holding.Company.Final,State,Census.Block.FIPS.Code,Technology.Code,Consumer,Max.Advertised.Downstream.Speed..mbps.,Max.Advertised.Upstream.Speed..mbps.,Business
,<int>,<int>,<int>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<dbl>,<int>,<int>,<dbl>,<dbl>,<int>
1,76132920,80632,1568880,GCI Communication Corp.,GCI Communication Corp.,GCI Holdings LLC,130534,GCI Holdings LLC,WA,5.304396e+14,60,0,0,0,1
2,45241346,86844,6945950,"T-Mobile USA, Inc.",T-Mobile,Deutsche Telekom AG,130403,"T-Mobile USA, Inc.",WA,5.306995e+14,70,1,25,3,1
3,829810,75896,18626853,"CenturyLink, Inc.",CenturyLink,"CenturyLink, Inc.",130228,"Lumen Technologies, Inc.",WA,5.303302e+14,12,1,100,10,1
4,10182979,77036,14694327,"STARTOUCH, INC.","StarTouch, Inc.","STARTOUCH, INC.",190342,"StarTouch, Inc.",WA,5.302997e+14,70,1,20,20,1
5,10213524,77036,14694327,"STARTOUCH, INC.","StarTouch, Inc.","STARTOUCH, INC.",190342,"StarTouch, Inc.",WA,5.306300e+14,70,1,20,20,1
6,16600087,78709,3768165,"COMCAST CABLE COMMUNICATIONS, LLC",Comcast,Comcast Corporation,130317,Comcast Corporation,WA,5.303303e+14,43,1,1200,35,1


In [7]:
tract$GEOID20 <- as.numeric(tract$GEOID20)

provider_data <- inner_join(tract, fcc_data, by = c("GEOID20" = "Census.Block.FIPS.Code"))

In [8]:
head(provider_data)

STATEFP COUNTYFP TRACTCE GEOID       NAME NAMELSAD          MTFCC FUNCSTAT
1 53      075      000602  53075000602 6.02 Census Tract 6.02 G5020 S       
2 53      075      000602  53075000602 6.02 Census Tract 6.02 G5020 S       
3 53      075      000602  53075000602 6.02 Census Tract 6.02 G5020 S       
4 53      075      000602  53075000602 6.02 Census Tract 6.02 G5020 S       
5 53      075      000602  53075000602 6.02 Census Tract 6.02 G5020 S       
6 53      075      000602  53075000602 6.02 Census Tract 6.02 G5020 S       
  ALAND    AWATER geometry                       ⋯
1 83019278 0      MULTIPOLYGON (((-117.1716 4... ⋯
2 83019278 0      MULTIPOLYGON (((-117.1716 4... ⋯
3 83019278 0      MULTIPOLYGON (((-117.1716 4... ⋯
4 83019278 0      MULTIPOLYGON (((-117.1716 4... ⋯
5 83019278 0      MULTIPOLYGON (((-117.1716 4... ⋯
6 83019278 0      MULTIPOLYGON (((-117.1716 4... ⋯
  Holding.Company.Name                 Holding.Company.Number
1 VSAT Systems, LLC.                   300167                
2 FIRST STEP INTERNET                  130500                
3 GCI Holdings LLC                     130534                
4 ViaSat, Inc.                         290111                
5 DishNET Holding, LLC                 130627                
6 Space Exploration Technologies Corp. 430076                
  Holding.Company.Final           State Technology.Code Consumer
1 VSAT Systems, LLC               WA    60              1       
2 First Step Internet, LLC        WA    70              1       
3 GCI Holdings LLC                WA    60              0       
4 ViaSat, Inc.                    WA    60              1       
5 Hughes Network Systems, LLC     WA    60              1       
6 Space Exploration Holdings, LLC WA    60              1       
  Max.Advertised.Downstream.Speed..mbps. Max.Advertised.Upstream.Speed..mbps.
1   2                                     1.3                                
2  25                                     4.0                                
3   0                                     0.0                                
4 100                                     3.0                                
5  25                                     3.0                                
6 100                                    10.0                                
  Business geometry                      
1 1        MULTIPOLYGON (((-117.1716 4...
2 0        MULTIPOLYGON (((-117.1716 4...
3 1        MULTIPOLYGON (((-117.1716 4...
4 1        MULTIPOLYGON (((-117.1716 4...
5 1        MULTIPOLYGON (((-117.1716 4...
6 1        MULTIPOLYGON (((-117.1716 4...

In [9]:
names(provider_data)

[1] "STATEFP"                               
 [2] "COUNTYFP"                              
 [3] "TRACTCE"                               
 [4] "GEOID"                                 
 [5] "NAME"                                  
 [6] "NAMELSAD"                              
 [7] "MTFCC"                                 
 [8] "FUNCSTAT"                              
 [9] "ALAND"                                 
[10] "AWATER"                                
[11] "INTPTLAT"                              
[12] "INTPTLON"                              
[13] "STATEFP20"                             
[14] "COUNTYFP20"                            
[15] "TRACTCE20"                             
[16] "BLOCKCE20"                             
[17] "GEOID20"                               
[18] "NAME20"                                
[19] "MTFCC20"                               
[20] "UR20"                                  
[21] "UACE20"                                
[22] "UATYPE20"                              
[23] "FUNCSTAT20"                            
[24] "ALAND20"                               
[25] "AWATER20"                              
[26] "INTPTLAT20"                            
[27] "INTPTLON20"                            
[28] "Logical.Record.Number"                 
[29] "Provider.ID"                           
[30] "FRN"                                   
[31] "Provider.Name"                         
[32] "DBA.Name"                              
[33] "Holding.Company.Name"                  
[34] "Holding.Company.Number"                
[35] "Holding.Company.Final"                 
[36] "State"                                 
[37] "Technology.Code"                       
[38] "Consumer"                              
[39] "Max.Advertised.Downstream.Speed..mbps."
[40] "Max.Advertised.Upstream.Speed..mbps."  
[41] "Business"                              
[42] "geometry"

In [9]:
provider_data <- provider_data %>% 
    select(
        tract = NAME, 
        company = DBA.Name, 
        is_consumer = Consumer, 
        is_business = Business, 
        max_d_mbps = Max.Advertised.Downstream.Speed..mbps.,
        max_u_mbps = Max.Advertised.Upstream.Speed..mbps.,
        technology_code = Technology.Code,
        provider_id = Provider.ID,
        GEOID
    )

In [10]:
head(provider_data)

,tract,company,is_consumer,is_business,max_d_mbps,max_u_mbps,technology_code,provider_id,GEOID,geometry
,<chr>,<chr>,<int>,<int>,<dbl>,<dbl>,<int>,<int>,<chr>,<MULTIPOLYGON [°]>
1,6.02,VSATSystemsLLC,1,1,2,1.3,60,76178,53075000602,MULTIPOLYGON (((-117.1716 4...
2,6.02,"First Step Internet, LLC",1,0,25,4.0,70,77236,53075000602,MULTIPOLYGON (((-117.1716 4...
3,6.02,GCI Communication Corp.,0,1,0,0.0,60,80632,53075000602,MULTIPOLYGON (((-117.1716 4...
4,6.02,Viasat Inc,1,1,100,3.0,60,79497,53075000602,MULTIPOLYGON (((-117.1716 4...
5,6.02,HughesNet,1,1,25,3.0,60,81097,53075000602,MULTIPOLYGON (((-117.1716 4...
6,6.02,Space Exploration Technologies Corp.,1,1,100,10.0,60,81199,53075000602,MULTIPOLYGON (((-117.1716 4...


In [11]:
provider_data %>% 
select(tract) %>% 
unique() %>% 
tally()

,n,geometry
,<int>,<POLYGON [°]>
1,181,"POLYGON ((-117.43 49.00046,..."


In [12]:
provider_data <- st_drop_geometry(provider_data)

In [14]:
head(provider_data, 20)

,tract,company,is_consumer,is_business,max_d_mbps,max_u_mbps,technology_code,provider_id
,<chr>,<chr>,<int>,<int>,<dbl>,<dbl>,<int>,<int>
1,6.02,VSATSystemsLLC,1,1,2,1.3,60,76178
2,6.02,"First Step Internet, LLC",1,0,25,4.0,70,77236
3,6.02,GCI Communication Corp.,0,1,0,0.0,60,80632
4,6.02,Viasat Inc,1,1,100,3.0,60,79497
5,6.02,HughesNet,1,1,25,3.0,60,81097
6,6.02,Space Exploration Technologies Corp.,1,1,100,10.0,60,81199
7,6.02,GCI Communication Corp.,0,1,0,0.0,60,80632
8,6.02,VSATSystemsLLC,1,1,2,1.3,60,76178
9,6.02,Space Exploration Technologies Corp.,1,1,100,10.0,60,81199


In [13]:
# write.csv(provider_data, "../data/provider_data.csv")